# E4. 멋진 작사가 만들기

## 데이터 읽기

In [1]:
import glob
import os
import re
import tensorflow as tf

from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []


# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you']


 ## 데이터 정제

In [2]:
# 정규표현식으로 문장 정제하기

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자 바꾸고 공백 없애기
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)  # 특수문자없애기
    sentence = re.sub(r'[" "]+', " ", sentence) # 공백 패턴 하나의 공백
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백
    sentence = sentence.strip() # 다시 양쪽 공백을 지우기
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호 및 특수문자 포함한 사이 글자 지우기
    #sentence = sentence.strip('Ooh''ooh') # Ooh삭제
    sentence = '<start> ' + sentence + ' <end>' #문장 시작에는 <start>, 끝에는 <end>를 추가
    
    
    return sentence

print(preprocess_sentence(raw_corpus[0]))   # 문장이 어떻게 필터링되는지 확인

<start> at first i was afraid <end>


- 길이가 짧은 단어
- 데이터 정제 참고 사이트: https://wikidocs.net/21693, https://velog.io/@ash3767/python-%EC%A0%95%EA%B7%9C%EC%8B%9D

In [3]:
#정제된 corpus 생성

corpus = []

for sentence in raw_corpus:
    #우리가 원하지 않는 문장을 건너뛰기
    if len(sentence) ==0: continue     #공백 문장 제외
    if len(sentence) >=15: continue    #길이가 15개가 넘는 것은 제외
    if sentence[-1] ==":": continue    #문장의 끝이 : 인 문장은 건너뛰기.
        
    # 정제를 하고 담아주기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
                  
                  
    #정제된 결과 15개만 확인
corpus[:15]

['<start> i grew strong <end>',
 '<start> i will survive <end>',
 '<start> yeah <end>',
 '<start> yeah , yeah <end>',
 '<start> and you see me <end>',
 '<start> well now go , <end>',
 '<start> oh not i , <end>',
 '<start> i will survive <end>',
 '<start> yeah , yeah <end>',
 '<start> up <end>',
 '<start> up <end>',
 '<start> yow , yow , yow <end>',
 '<start> i don t wanna <end>',
 '<start> i don t wanna <end>',
 '<start> yeah , ho , yow <end>']

In [4]:
#토큰화

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,   # 전체 단어의 개수 
        filters=' ',       # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # padding으로 입력 데이터의 시퀀스 길이를 일정하게 맞추기 (maxlen 15로 설정) 
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


In [5]:
print(tensor[:3,:10]) #tokenizer에 구축된 단어 사전의 인덱스
tensor.shape

[[  2   4 910 483   3   0   0   0   0   0]
 [  2   4 116 484   3   0   0   0   0   0]
 [  2  13   3   0   0   0   0   0   0   0]]


(12147, 15)

In [6]:
# 단어 사전이 어떻게 구축되었는지 확인 방법
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break     

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : .
7 : oh
8 : you
9 : it
10 : me


In [7]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

# 소스는 2(<start>)에서 시작해서 3(<end>)으로 끝난 후 0(<pad>)로 채워져 있습니다. 

[  2   4 910 483   3   0   0   0   0   0   0   0   0   0]
[  4 910 483   3   0   0   0   0   0   0   0   0   0   0]


# 평가 데이터셋 분리

In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,test_size=0.2)
print('enc_train 개수: ',len(enc_train), ',dec_val 개수:', len(dec_val))

enc_train 개수:  9717 ,dec_val 개수: 2430


In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (9717, 14)
Target Train: (9717, 14)


In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# 인공지능 만들기

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        #embedding은 문자를 숫자로 바꾸는 것
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 # embedding_size 는 워드 벡터의 차원수, 단어가 추상적으로 표현되는 크기, 너무 크면 혼란이 된다.
hidden_size = 1024 # LSTM 레이어의 hidden state 의 차원수
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


 #데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)


model.summary()
#모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape를 특정할 수 없는 것
# 모델의 파라미터 사이즈는 측정됩니다. 대략 29million 

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [12]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
47/47 [==============================] - 17s 313ms/step - loss: 4.1506
Epoch 2/30
47/47 [==============================] - 15s 312ms/step - loss: 1.4817
Epoch 3/30
47/47 [==============================] - 15s 316ms/step - loss: 1.2245
Epoch 4/30
47/47 [==============================] - 15s 315ms/step - loss: 1.1368
Epoch 5/30
47/47 [==============================] - 15s 319ms/step - loss: 1.1071
Epoch 6/30
47/47 [==============================] - 15s 319ms/step - loss: 1.0765
Epoch 7/30
47/47 [==============================] - 15s 322ms/step - loss: 1.0582
Epoch 8/30
47/47 [==============================] - 15s 324ms/step - loss: 1.0293
Epoch 9/30
47/47 [==============================] - 15s 325ms/step - loss: 1.0084
Epoch 10/30
47/47 [==============================] - 15s 326ms/step - loss: 0.9898
Epoch 11/30
47/47 [==============================] - 15s 326ms/step - loss: 0.9740
Epoch 12/30
47/47 [==============================] - 15s 327ms/step - loss: 0.9552
Epoch 13/30
4

# 평가하기

In [13]:
def generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
      # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    # for문을 통해서 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [14]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i", max_len=20)

'<start> i m a survivor <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> you ", max_len=20)

'<start> you re the one <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> summer ", max_len=20)

'<start> summer s all right <end> '

## 회고
- text에서는 데이터 정제하는 것이 중요 